In [ ]:
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import os
import sys
sys.path.append("../")
import random
from aif360.algorithms.inprocessing.gerryfair_classifier import *
from aif360.algorithms.inprocessing.gerryfair.clean import *
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import *
from aif360.algorithms.inprocessing.gerryfair.auditor import *
from sklearn import svm
from sklearn import tree
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
import pickle
import matplotlib.pyplot as plt
import random



We first demonstrate how to instantiate a `Model`, `train` it with respect to rich subgroup fairness, and `predict` the label of a new example. We remark that when we set the `print_flag = True` at each iteration of the algorithm we print the error, fairness violation, and violated group size of most recent model. The error is the classification error of the classifier. At each round the Learner tries to find a classifier that minimizes the classification error plus a weighted sum of the fairness disparities on all the groups that the Auditor has found up until that point. By contrast the Auditor tries to find the group at each round with the greatest rich subgroup disparity with respect to the Learner's model. We define `violated group size` as the size (as a fraction of the dataset size) of this group, and the `fairness violation` as the `violated group size` times the difference in the statistical rate (FP or FN rate) on the group vs. the whole population. 

In the example below we set `max_iterations=5000` which is in-line with time to convergence observed in [the rich subgroup fairness empirical paper](https://arxiv.org/abs/1808.08166), but advise that this can be highly dataset dependent. Our target $\gamma$-disparity is `gamma = .001`, our statistical rate is false positive rate or `FP`, and our weighted regression oracle is linear regression. We observe that the unconstrained (with no fairness constraint) classifier has error $.185$ and $\gamma$-disparity $.022$. After $5000$ iterations we obtain a classifier that $\gamma$-fair, and has error $.230$. We note that we converge to a fair, and almost as accurate classifier after just a hundred iterations.


In [6]:
random.seed(1)
C = 100
print_flag = True
gamma = .001
max_iterations = 150

fair_model = Model(C=C, printflag=print_flag, gamma=gamma, fairness_def='FP',
             max_iters=max_iterations, heatmapflag=False)
data_set = load_preproc_data_adult(sub_samp=200)

# fit method
communities_all_errors, communities_violations = fair_model.fit(data_set,
                                                 early_termination=True, return_values=True)
# predict method
dataset_yhat = fair_model.predict(data_set)



iteration: 1
most accurate classifier error: 0.215, most accurate class unfairness: 0.01868421052631579, violated group size: 0.405
iteration: 2
error: 0.2275, fairness violation: 0.009342105263157895, violated group size: 0.405
iteration: 3
error: 0.23166666666666672, fairness violation: 0.0062280701754385956, violated group size: 0.355
iteration: 4
error: 0.23375, fairness violation: 0.004671052631578947, violated group size: 0.405
iteration: 5
error: 0.235, fairness violation: 0.003736842105263158, violated group size: 0.405
iteration: 6
error: 0.23583333333333328, fairness violation: 0.0031140350877192986, violated group size: 0.355
iteration: 7
error: 0.23642857142857146, fairness violation: 0.0026691729323308276, violated group size: 0.355
iteration: 8
error: 0.236875, fairness violation: 0.0023355263157894745, violated group size: 0.355
iteration: 9
error: 0.23722222222222222, fairness violation: 0.0020760233918128666, violated group size: 0.405
iteration: 10
error: 0.2375, fair

iteration: 76
error: 0.23967105263157895, fairness violation: 0.0002458448753462605, violated group size: 0.405
iteration: 77
error: 0.23967532467532468, fairness violation: 0.00024265208475734802, violated group size: 0.405
iteration: 78
error: 0.2396794871794872, fairness violation: 0.00023954116059379221, violated group size: 0.355
iteration: 79
error: 0.23968354430379749, fairness violation: 0.00023650899400399738, violated group size: 0.355
iteration: 80
error: 0.2396875, fairness violation: 0.00023355263157894747, violated group size: 0.405
iteration: 81
error: 0.23969135802469135, fairness violation: 0.0002306692657569851, violated group size: 0.355
iteration: 82
error: 0.2396951219512195, fairness violation: 0.00022785622593068044, violated group size: 0.405
iteration: 83
error: 0.2396987951807229, fairness violation: 0.00022511097019657582, violated group size: 0.405
iteration: 84
error: 0.23970238095238094, fairness violation: 0.00022243107769423563, violated group size: 0.35

iteration: 149
error: 0.23983221476510067, fairness violation: 0.00012539738608265642, violated group size: 0.355


In [ ]:

# output heatmap (brute force)
# replace None with the relative path if you want to save the plot
fair_model.heatmapflag = True
fair_model.save_heatmap(fair_model.max_iters, data_set, dataset_yhat.labels, None, None)






In [ ]:
# auditing a classifier for unfairness
# instantiate auditor
auditor = Auditor(data_set, 'FP')
group = auditor.get_group(dataset_yhat.labels, auditor.get_baseline(array_to_tuple(data_set.labels), array_to_tuple(dataset_yhat.labels)))
print('gamma disparity: {}'.format(group.weighted_disparity))






In [35]:
# run & create pareto curves
fair_model.max_iters = 10
fair_model.printflag = False
gamma_list = [.01, .02, .03, 1.0]
fair_model.pareto(data_set, gamma_list)



iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
iteration: 3
iteration: 4
iteration: 5


([0.2227777777777778, 0.21055555555555558, 0.185, 0.185],
 [0.004084967320261439,
  0.0062636165577342065,
  0.02205882352941177,
  0.02205882352941177],
 [0.0, 0.0057446808510638455, 0.018191489361702128, 0.018191489361702128])

In [ ]:
def multiple_classifiers_pareto(dataset, gamma_list=[0.002, 0.005, 0.01, 0.02, 0.05, 0.1], save_results=False):

    ln_predictor = linear_model.LinearRegression()
    svm_predictor = svm.LinearSVR()
    tree_predictor = tree.DecisionTreeRegressor(max_depth=3)
    kernel_predictor = KernelRidge(alpha=1.0, gamma=1.0, kernel='rbf')
    predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': 10},
                      'SVR': {'predictor': svm_predictor, 'iters': 10},
                      'Tree': {'predictor': tree_predictor, 'iters': 10},
                      'Kernel': {'predictor': kernel_predictor, 'iters': 10}}

    results_dict = {}

    for pred in predictor_dict:
        print('Curr Predictor: {}'.format(pred))
        predictor = predictor_dict[pred]['predictor']
        max_iters = predictor_dict[pred]['iters']
        fair_clf = Model(C=100, printflag=True, gamma=1, predictor=predictor, max_iters=max_iters)
        fair_clf.set_options(max_iters=max_iters)
        errors, fp_violations, fn_violations = fair_clf.pareto(dataset, gamma_list)
        results_dict[pred] = {'errors': errors, 'fp_violations': fp_violations, 'fn_violations': fn_violations}
        plt.plot(errors, fp_violations, label=pred)

    if save_results:
        pickle.dump(results_dict, open('results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb'))

    plt.xlabel('Error')
    plt.ylabel('Unfairness')
    plt.legend()
    plt.title('Error vs. Unfairness\n(Communities & Crime Dataset)')
    plt.show()
    
    
multiple_classifiers_pareto(data_set)

In [ ]:

def fp_vs_fn(dataset, fair_model, gamma_list):
    fp_auditor = Auditor(dataset, 'FP')
    fn_auditor = Auditor(dataset, 'FN')
    fp_violations = []
    fn_violations = []
    for g in gamma_list:
        fair_model.set_options(gamma=g)
        fair_model.fit(dataset)
        predictions = (fair_model.predict(dataset)).labels
        predictions_inv = [abs(1 - p) for p in predictions]
        _, fp_diff = fp_auditor.audit(predictions)
        _, fn_diff = fn_auditor.audit(predictions_inv)
        fp_violations.append(fp_diff)
        fn_violations.append(fn_diff)

    print((fp_violations, fn_violations))

    plt.plot(fp_violations, fn_violations, label='communities')
    plt.xlabel('False Positive Disparity')
    plt.ylabel('False Negative Disparity')
    plt.legend()
    plt.title('FP vs FN Unfairness')
    plt.show()
    
C = 10
print_flag = True
gamma = .01
max_iterations = 10
fair_def = 'FP'
fair_model = Model(C=C, printflag=print_flag, gamma=gamma, fairness_def=fair_def,
             max_iters=max_iterations)
data_set = load_preproc_data_adult(sub_samp=50)
gamma_list = [0.001, 0.002, 0.003, 0.004, 0.005, 0.0075, 0.01, 0.02, 0.03, 0.05]
fp_vs_fn(data_set, fair_model, gamma_list)

NameError: name 'load_preproc_data_adult' is not defined